## **LangChain -  FAISS**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [ ]:
# 1) load PDF
pdf_path = 'Hayat-Qulub-Alama-Majlisi.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [ ]:
print(loader)

In [ ]:
# 2) Split into chunks 
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)

In [ ]:
# 3-A) Create Embeddings - Sentenace Transformer Embeddings
embeddings = HuggingFaceBgeEmbeddings(model_name='all-MiniLM-L12-v2', model_kwargs={'device':'cuda'})
embeddings

In [ ]:
# 4) Store Vector Embeddings
vectordb = FAISS.from_documents(chunks, embeddings)
vectordb.save_local("faiss_all_minilm")

In [ ]:
# 5) Retrieve 
retrieve = vectordb.as_retriever(search_type = 'similarity', search_kwargs={"k":10})
retrieve

In [ ]:
from dotenv import load_dotenv
# ---------- Load env var ----------
load_dotenv()

API_KEYS = os.getenv("GEMINI_API_KEYS", "").split(",")
API_KEYS = [k.strip() for k in API_KEYS if k.strip()]
current_key_index = 0
ACCESS_KEY = os.getenv("APP_ACCESS_KEY")

if not API_KEYS:
    print("❌ No Gemini API keys found. Check .env file.")
    raise Exception("Missing GEMINI_API_KEYS")

In [ ]:
# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
If the answer is not in the documents, say "I don’t know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=system_prompt
)

In [ ]:
# 6) LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    api_key=os.getenv('GEMINI_KEY_KEY', '')
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retrieve,
    return_source_documents=False,
    chain_type_kwargs = {"prompt":prompt}
)

In [ ]:
# 7) Ask
query = "Why Adam was named Adam? Give reference to it as well"
res = qa.run(query)
print(res)

In [ ]:
q1 = "Why Adam was named Adam? Explain in details"
res = qa.run(q1)
print(res)

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

## **Pinecone Storage**

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ_V2")
index = pc.Index(INDEX_NAME)


In [ ]:
index.delete(delete_all=True)

### **Vectorinze Single PDF**

In [ ]:
# ---------- begin of pipelines ----------
# load pdf
pdf_path = 'Hayat-al-Qulub-Vol-1.pdf'
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

In [ ]:
from langchain.schema import Document
import json
with open("hadiths_updated.json", "r", encoding="utf-8") as f:
    data = json.load(f)   # expecting a list of {"quote_no":1,"quote":"..."} or a wrapper object

# if JSON is a wrapper like {"quotes": [...]}
if isinstance(data, dict) and "hadith_text" in data:
    items = data["quotes"]
else:
    items = data

# docs = []

# for item in items:
#     text = item.get("hadith_text") or item.get("text") or item.get("body") or ""
#     meta = {"hadith_no": item.get("hadith_no"), **{k:v for k,v in item.items() if k not in ("hadith_text","text","body")}}
#     docs.append(Document(page_content=text, metadata=meta))


In [ ]:
docs = []
for item in items:
    hadith_no = item.get("hadith_no")
    text = item.get("hadith_text","").strip()
    # append ID into the content
    content = f"{text}\n\n[HADITH_NO: {hadith_no}]"
    meta = {k:v for k,v in item.items() if k not in ("hadith_text",)}
    docs.append(Document(page_content=content, metadata=meta))

In [ ]:
docs[0]

In [ ]:
docs[1]

In [ ]:
# split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_documents(docs)
chunks

In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)

In [ ]:
# Store in vector database - Pinecone
vector_db = PineconeVectorStore.from_documents(documents=chunks, embedding=embeddings, index_name=INDEX_NAME)

In [ ]:
# Store in vector database - FAISS
vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("Hayat-Qulub-Alama-Majlisi-faiss-index")

### **Vectorize multiple PDFs**

In [ ]:
#  Store all 3 PDFs in vector database - Pinecone
pdfs = [
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-1.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-2.pdf',
    'hayatal-qulub-pdfs/Hayat-al-Qulub-Vol-3.pdf'
]

all_chunks = []
for i, pdf in enumerate(pdfs):
    loader = PyMuPDFLoader(pdf)
    docs = loader.load()
    print(f"Loaded {i} documents")
    
    # split text into chunks
    print(f"Splitting {i} documents into chunks...")
    splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    chunks = splitter.split_documents(docs)
    print(f"splitted {i} documents Successfully! ")
    print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

    all_chunks.extend(chunks)
    print(f"Chunk list filled")
    print(f"Total chunks so far: {len(all_chunks)}")
    


In [ ]:
# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cuda"}
)


In [ ]:
# Store in vector database - Pinecone
vector_db = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
pinecone_store = vector_db

In [ ]:
pinecone_store = vector_db

### **Vectorize multiple PDFs - functions**

In [ ]:
def process_pdf_into_chunks(pdfs):
    pdfs = list(pdfs)
    all_chunks = []
    for i, pdf in enumerate(pdfs):
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} documents")
        
        # split text into chunks
        print(f"Splitting {i} documents into chunks...")
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"splitted {i} documents Successfully! ")
        print(f"Loaded {len(chunks)} chunks from {i} document {pdf}")

        all_chunks.extend(chunks)
        print(f"Chunk list filled")
        print(f"Total chunks so far: {len(all_chunks)}")
    
    return all_chunks

def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
    # create embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model, model_kwargs={"device": "cuda"}
    )
    print(f"Created embeddings using model: {embedding_model}")
    
    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(documents=all_chunks, embedding=embeddings, index_name=INDEX_NAME)
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store


### **Retrieve QA**

In [ ]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cpu"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = """
You are an Islamic history assistant. 
Always answer in a respectful and storytelling way. 
You have to understand the context deeply and answer accordingly.
If the answer is not in the documents, say "I don't know based on my knowledge."
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())


In [ ]:
# Ask a question
query = "Why Adam was named Adam? Give reference to it as well"
res = qa_chain.invoke(query)

print(res)

In [ ]:
query1 = """
Kisne likh hai Hayatal Qulub? kiske bare me hai ye kitab? 
kitni jildhe hai iske aur kya har ek jildh ko short me smjha sakte ho?
"""
res2 = qa_chain.invoke(query1)
print(res2)

In [ ]:
# quick check for any embedding wrapper you plan to use
vec = embeddings.embed_query("test")
print("type:", type(vec))
print("len:", len(vec))   # MUST be 1024 to match your Pinecone index

## **Sulaym ibne Qays RAG**

### **RAG Pipeline**

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeBaseClient
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ")
index = pc.Index(INDEX_NAME)

In [ ]:
def process_pdf_into_chunks(pdfs: list):
    print(f'Count of Documents - {len(pdfs)}')

    all_chunks = []
    for i, pdf in enumerate(pdfs):
        i=+1
        loader = PyMuPDFLoader(pdf)
        docs = loader.load()
        print(f"Loaded {i} document successfully")

        # split into chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        chunks = splitter.split_documents(docs)
        print(f"Splitted {i} documment successfully!")

        all_chunks.extend(chunks)
        print(f"Total chunks so far: {len(all_chunks)}")

    return all_chunks
    
def store_chunks_in_pinecone(all_chunks, embedding_model, INDEX_NAME):
     #create embeddings

    embeddings = HuggingFaceEmbeddings(
        model_name = 'BAAI/bge-large-en-v1.5', model_kwargs={"device": "cuda"} 
        )
    print(f"Created embeddings using model: {embedding_model}")

    # Store in vector database - Pinecone
    pinecone_store = PineconeVectorStore.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        index_name=INDEX_NAME
    )
    print(f"Stored {len(all_chunks)} chunks in Pinecone index: {INDEX_NAME}")
    return pinecone_store
    

In [ ]:
chunks = process_pdf_into_chunks(pdfs=['kitab_sulaim_ibn_qays_al-hilaali.pdf'])

In [ ]:
vector_store = store_chunks_in_pinecone(chunks, 'BAAI/bge-large-en-v1.5', INDEX_NAME)

### **Retrieve QA**

In [3]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone as PineconeBaseClient
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Markdown

from system_prompt import system_prompt_text
load_dotenv()

# Setup Pinecone
pc = PineconeBaseClient(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME_SQ_V2")
index = pc.Index(INDEX_NAME)

# create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5", model_kwargs={"device": "cpu"}
)

pinecone_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME, 
    embedding=embeddings)

# Create retriever from Pinecone
retrieve = pinecone_store.as_retriever(
    search_type='similarity', 
    search_kwargs={"k": 10}
)

# System Prompt
system_prompt = system_prompt_text

prompt = ChatPromptTemplate.from_template(system_prompt)

# LLM and QA
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash',
    temperature=0,
    api_key=os.getenv('GEMINI_API_KEY')
)

# llm = ChatOpenAI(
#     model_name='gpt-4o', 
#     temperature=0,
#     api_key=os.getenv('OPEN_AI_KEY')
# )

def format_docs(docs):
    blocks = []
    for d in docs:
        hadith_no = d.metadata.get("hadith_no", "UNKNOWN")
        page_start = d.metadata.get("page_start")
        # ensure hadith_no is integer-like for neatness
        try:
            hadith_tag = f"[HADITH_NO:{int(hadith_no)}]"
        except:
            hadith_tag = f"[HADITH_NO:{hadith_no}]"
        # clean page markers inside content (optional)
        content = d.page_content.replace("(Page", "").replace("Page", "")
        content = content.strip()
        blocks.append(f"{hadith_tag} | PAGE:{page_start}\n{content}")
    return "\n\n".join(blocks)


input = {"context": retrieve | format_docs, 
      "question": RunnablePassthrough()
      }


qa_chain = ( input| prompt | llm | StrOutputParser())

def ask(question: str) -> str:
    """Ask a question and get an answer from the RAG system."""
    result = qa_chain.invoke(question)
    return result

d:\Sadiq\rag-for-hq-books\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Sadiq\rag-for-hq-books\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [2]:
# # debug inspect retrieved docs
# retrieved = retrieve.get_relevant_documents(query)
# for i, d in enumerate(retrieved[:8]):
#     print(f"--- RESULT {i} ---")
#     print("metadata keys:", list(d.metadata.keys()))
#     print("hadith_no:", d.metadata.get("hadith_no"))
#     print("page_no:", d.metadata.get("page_start") or d.metadata.get("page_no") or d.metadata.get("page"))
#     print("first 200 chars:", d.page_content[:200].replace("\n"," "))
#     print()


In [3]:
# res = index.fetch(ids=["1"])   # try an id you expect to exist
# print(res)   # look under 'vectors' -> id -> metadata

In [4]:
from IPython.display import display, Markdown
# Ask a question
query = "Few people attacked the house of Lady Fatimah (SA) and burned the door. what was the entire incident? Explain in details with hadith no."
#res = qa_chain.invoke(query)
res = ask(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that...

After the Holy Prophet (SAWA) passed away, Umar gathered people to force Imam Ali (AS) to pledge allegiance to Abu Bakr. They went to Imam Ali's (AS) house, where Lady Fatimah (SA) and their two sons were. Umar ordered his companions to collect wood and pile it around the house [HADITH_NO:4].

Umar then shouted loudly, so that Imam Ali (AS) and Lady Fatimah (SA) could hear: "By God, O Ali, come out and pay allegiance to the caliph of the Holy Prophet of Allah, otherwise we will burn your house" [HADITH_NO:4].

Lady Fatimah (SA) asked: "O Umar, what do you have to do with us?" [HADITH_NO:4].

Umar replied: "Open the door, otherwise we will burn your house" [HADITH_NO:4], [HADITH_NO:48].

Lady Fatimah (SA) said: "O Umar, are you not afraid of Allah and are you entering our house?" [HADITH_NO:4].

Umar refused to return. He asked for fire to be brought and he set the door on fire, then he pushed it and entered. Lady Fatimah (SA) came in front and screamed loudly: "O Father, O Prophet of Allah" [HADITH_NO:4], [HADITH_NO:48].

According to the traditions, Janabe Sulaym ibne Qays narrated from Salman that...

Salman said: "Yes, by God, when she did not even have a chaddar over her. So she screamed: “O Father, O Prophet of Allah, Abu Bakr and Umar behaved so badly after you, while your eyes have not even closed in the grave” and she was saying this loudly" [HADITH_NO:4].

According to the traditions, Janabe Sulaym ibne Qays narrated that...

Umar raised his sword with the shield and hit her on the side. She screamed [HADITH_NO:4].

References: [HADITH_NO:4], [HADITH_NO:48]

In [3]:
query = "Explain the Hadith no 4 exactly in detail"
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that after the demise of the Prophet Muhammad (PBUH), he was present when a man from the Ansar spoke to a group of people, including himself, about the events that transpired. The man said:

"When the Messenger of Allah (PBUH) passed away, Ali (AS) was occupied with his burial. A group of people gathered in the Saqifa of Bani Sa'ida to exclude him from the affair (of leadership). When Ali (AS) finished with him (the Prophet (PBUH)), he sat at home and gathered the Quran, for he had vowed to Allah that he would not put on his outer garment except for prayer until he had gathered it. Then, Abu Bakr sent word to him, 'Come and pledge allegiance.' But Ali (AS) sent back to him, 'I am occupied.' Abu Bakr sent word to him again, 'Come, or I will come to you and fight you.' Then Umar (LA) came with a group and set fire to Ali's (AS) door. Fatima (SA) came out and said, 'O Umar, what is this?' He said, 'Why don't people enter for allegiance?' Then they took Ali (AS) forcefully and brought him to Abu Bakr." [HADITH_NO:4]

The man from the Ansar continued, "Ali (AS) said to Abu Bakr, 'O Abu Bakr, you are more entitled to this affair than I am. I will obey you and give you allegiance, but tell me, O Muhajirun, is there anyone among you who is closer to the Messenger of Allah (PBUH) than I am? Or who has more knowledge of the Quran and the Sunnah than I do? Or who is more deserving of this affair than I am?' They said, 'No, by Allah.' Then Ali (AS) said, 'I ask you by Allah, O Muhajirun, did you hear the Messenger of Allah (PBUH) say on the Day of Ghadir Khumm, "Whoever I am his master (Mawla), then Ali is his master (Mawla). O Allah, befriend whoever befriends him and be an enemy to whoever is his enemy; and support whoever supports him and abandon whoever abandons him?"' They said, 'Yes, by Allah, we heard it.' Ali (AS) said, 'Then why did you dispute with me over this affair?' They said, 'We feared that you would not be able to handle it.'" [HADITH_NO:4]

The man from the Ansar further narrated, "Then Ali (AS) said, 'By Allah, I am more capable of handling it than you are. But I will obey you as long as I live.' Then Ali (AS) gave allegiance to Abu Bakr." [HADITH_NO:4]

References: [HADITH_NO:4]

In [13]:
query = "Who are those 3 types of people who understand and accept the Ahle bait (as)?"
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that there are three types of people: - a believer who knows our rights, believes in what we say, and in every matter keeps us ahead. This is the person who is saved and is honest and is Allah’s friend. Whoever established enmity with us and kept away from us, curses us and considers our blood permitted, and denies our rights and has made keeping away from us religion - this person is an infidel, an idolater and cursed. The one who chooses that in which there is no dispute and whatever he finds difficulty in, he returns its knowledge to Allah, even if it is our Wilayat, and does not bear enmity towards us, we are hopeful for him and his decision is in Allah’s hands [HADITH_NO:42].

References: [HADITH_NO:42]

In [15]:
query = "Who are those 3 types of people who understand and accept the Ahle bait (as) repsond me in urdu written in english?"
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated from Imam Ali (AS) that...

Imam Ali (AS) ne farmaya: "Beshak, teen tarah ke log hain: - ek momin jo hamare huqooq ko jaanta hai, hum jo kehte hain us par yaqeen rakhta hai, aur har maamle mein humein aage rakhta hai. Yeh woh shakhs hai jo bachaya gaya hai aur woh sachcha aur Allah ka dost hai. Jo koi humse dushmani karta hai aur humse door rehta hai, hum par laanat karta hai aur hamare khoon ko jaaiz samajhta hai, aur hamare huqooq ka inkaar karta hai aur humse door rehne ko mazhab bana liya hai - yeh shakhs kafir, but-parast aur mal'oon hai. Woh jo us cheez ko chunta hai jis mein koi ikhtilaaf nahi hai aur jis cheez mein use mushkil hoti hai, woh uske ilm ko Allah ki taraf lauta deta hai, chahe woh hamari Wilayat hi kyun na ho, aur humse dushmani nahi rakhta, hum uske liye ummeedwar hain aur uska faisla Allah ke haath mein hai." [HADITH_NO:42]

References: [HADITH_NO:42]

In [5]:
# Ask a question
query = """
Give me 5 merits of Imam Ali (AS) in Islam with references.
"""
res = qa_chain.invoke(query)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that:

1.  Imam Ali (AS) was the first to accept Islam and was more knowledgeable than all the companions of the Holy Prophet (SAWA) [HADITH_NO:58].
2.  Imam Ali (AS) tolerated more difficulties fighting in the way of Allah and defending from enemies and saving the Holy Prophet (SAWA) at the cost of his own life [HADITH_NO:58].
3.  Imam Ali (AS) is the most knowledgeable in the Book of Allah and the traditions of the Holy Prophet (SAWA) [HADITH_NO:58].
4.  The Holy Prophet (SAWA) said to Imam Ali (AS): "O Ali, you are brother, you are friend, you are successor, you are vizier, you are caliph in family and wealth and at all time when I am not there. And you have the same status to me that I have with my God. You are caliph in my Ummah. Your friend is my friend, and your enemy is my enemy. You are Amirul Mumineen and Sayyidul Muslimeen after me" [HADITH_NO:40].
5.  The Holy Prophet (SAWA) said: "Do not upset me concerning my brother Ali because he is Amirul Mumineen, Sayyidul Mursleen and is the bearer of Livai Hamd (Standard bearer) and on the Day of Judgement he is Qaedul Gurril Muhajjileen (the leader of those whose forehead will be shining). Allah will nominate him on Sirat (The Path) and he will divide the Hell. He will enter his friends in Heaven and enemies in Hell" [HADITH_NO:24].

References: [HADITH_NO:58], [HADITH_NO:40], [HADITH_NO:24]

In [5]:
from IPython.display import display, Markdown
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that...

After the Holy Prophet (SAWA) passed away, Umar gathered people to force Imam Ali (AS) to pledge allegiance to Abu Bakr. They went to Imam Ali's (AS) house, where Lady Fatimah (SA) and their two sons were. Umar ordered his companions to collect wood and pile it around the house [HADITH_NO:4].

Umar then shouted loudly, so that Imam Ali (AS) and Lady Fatimah (SA) could hear: "By God, O Ali, come out and pay allegiance to the caliph of the Holy Prophet of Allah, otherwise we will burn your house" [HADITH_NO:4].

Lady Fatimah (SA) asked: "O Umar, what do you have to do with us?" [HADITH_NO:4].

Umar replied: "Open the door, otherwise we will burn your house" [HADITH_NO:4], [HADITH_NO:48].

Lady Fatimah (SA) said: "O Umar, are you not afraid of Allah and are you entering our house?" [HADITH_NO:4].

Umar refused to return. He asked for fire to be brought and he set the door on fire, then he pushed it and entered. Lady Fatimah (SA) came in front and screamed loudly: "O Father, O Prophet of Allah" [HADITH_NO:4], [HADITH_NO:48].

According to the traditions, Janabe Sulaym ibne Qays narrated from Salman that...

Salman said: "Yes, by God, when she did not even have a chaddar over her. So she screamed: “O Father, O Prophet of Allah, Abu Bakr and Umar behaved so badly after you, while your eyes have not even closed in the grave” and she was saying this loudly" [HADITH_NO:4].

According to the traditions, Janabe Sulaym ibne Qays narrated that...

Umar raised his sword with the shield and hit her on the side. She screamed [HADITH_NO:4].

References: [HADITH_NO:4], [HADITH_NO:48]

In [6]:
# Ask a question
query = "Tell me about the 73rd sect which will go to paradise with references."
res = qa_chain.invoke(query)

#print(res)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated from Ali ibn Abi Talib (AS) that: “The Ummah will be separated in 73 sects, of which 72 will go to Hell, and 1 will go to Heaven. [HADITH_NO:7]

According to the traditions, Janabe Sulaym ibne Qays narrated that Ali (AS) was saying to the leader of Jews: “How many sects have happened amongst you?” He replied: “This many and this many.” Ali (AS) said: “You have told a lie.” Then he turned to people and said: “By God, if a cushion is kept for me I would make a judgment based on their Torah among the people of Torah and on their Injil amongst the people of Injil and on Quran amonst people of Quran. Jews have become 71 sects - 70 sects from them will be in Hell, 1 sect will go to Heaven and this is the one that obeyed Yush’a ibn Noon, the trustee of Musa and Christians are divided into 72 sects. 71 from them will go to Hell and 1 in Heaven and this is the one who obeyed Sham’un, the trustee of Isa (AS). This Ummah will have 73 sects - 72 from it will go to Hell and 1 will go to Heaven, and this the sect that obeyed the successor of Muhammad (SAW). [HADITH_NO:32]

According to the traditions, Janabe Sulaym ibne Qays narrated that he heard the Holy Prophet (SAWA) say: “The Jews have 71 sects, from which 70 will go to Hell and one will go to Heaven and it is the sect that followed the wasi and Musa (AS). The Christians have 72 sects, 71 will go to Hell and one will go to Heaven and it is that sect who followed Isa’s wasi, and my Ummah will have 73 sects, 72 will go to Hell and one will go to Heaven and that sect is the one that follows my wasi.” [HADITH_NO:65]

According to the traditions, Janabe Sulaym ibne Qays narrated that Ali ibn Abi Talib (AS) said: “From the 73, 13 will show their love towards us Ahlulbayt, from which 1 will go to Heaven and 12 will go to Hell. The sect that will be saved will be the one which is guided, which is hopeful, which has Iman, obeys command (bows his head), is agreeable and intelligent. It is he that trusts me and bows his head to my command, obeys my command, chooses to keep away from my enemy, loves me, keeps enmity with my enemy, has recognised my rights, and Imamat and has realised my obedience compulsory through the Book of Allah and the Tradtion of the Holy Prophet, and has not become an apostate, and has not doubted, because Allah has enlightened his heart with the knowledge of our rights and has recognised our merits and all this has taken those 73 sects as those who are deniers, enemies, who came forward and called people towards their Deen. From these 1 will be the one that has chosen Allah’s Deen, and 72 will be those who chose Shaytan’s deen and will love the deen that they accepted. They will stay away from those who oppose their deen. But those who believe in Tawheed and have brought Iman on the Holy Prophet (S.A.W) but do not know our Wilayat and do not know the misguidance of our enemies, have not made halal or haram, and have chosen those things in which there is no disagreement, and which Allah commanded. In matters where there is disagreement in the Ummah, they stay away, not sure of whether it has been commanded or not – so they have not chosen to be enemy or have not made lawful or unlawful, have no knowledge and whatever difficulty they come across, they leave it to Allah. So they will be saved. This sect is in-between the faithful and idolater and majority of people are idolaters will go to Hell without questions. Reckoning will only be for those types of people who are in between the faithful and idolaters, and muallafatul qulub (those whose hearts can be attracted) and those who are mukhtnfah (who have a little knowledge) and those who, along with good deeds, have done bad also and are with weak faith who are confused about kufr and shirk and do not really know whether to ebar enmity and will not even choose that Path which will make them Mumin or Aarif. These people will be from Ashab-‘Aaraf. These will be those at Allah’s discretion. If Allah sends him to Hell, then that will be due to sins, and if He forgives then it will be His Mercy.” [HADITH_NO:7]

References: [HADITH_NO:7], [HADITH_NO:32], [HADITH_NO:65]

In [ ]:
# Ask a question
query = "Tell me bout diff between imaan n islam as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

In [7]:
# Ask a question
query = "Tell me bout the accursed clan as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that the Holy Prophet (S.A.W) said to Imam Ali (AS): "you are my vizier, and you are my inheritor and you will fight on my tradition. You have the same status to me as Haroon had to Musa and for you the best footsteps of Haroon are present. When his people made him weak and over powered him it was not far that they kill him. So be patient with oppression of Quraysh and their over powering you because the chests of community is filled with malice and the enmity of Badr and Ohad. Musa had ordered Haroon when he was made caliph in his community that if they become misguided and he gets helpers then he could fight against the community. If he does not find helpers then restrain and save blood and do not create disunity among them. So you also do the same. If you get helpers then fight against them, if you do not get helpers then restrain your hand and save your blood because if you oppose them they will kill you. If they listen to you and obey you then guide them on Truth and you should know that if you there will be seven rulers from the children of Abul Aas, and from children of Marwan there will be five. This is how there will be twelve imams whom the Holy Prophet (S.A.W) had seen climbing and stepping down from the pulpit in a manner that monkeys go up and down. These people will return the Ummah of the Holy Prophet (S.AW) from the Deen of Allah in reverse.  
No doubt these people on the Day of Judgement will be punished most, surely, Allah will take caliphate out from this through those black standards that will be coming from the East. Allah will, through them, humiliate them and will kill them under every stone. And a person from your children who is unfortunate, cursed, rude, unpleasant, whose heart will be on the wrong side, hot-tempered and cruel - Allah has taken away softness and kindness from his heart, and his maternal uncle will be from the family of Kalb - I can see him and if I wished I could tell his name as well as his qualities, and how [HADITH_NO:25].

References: [HADITH_NO:25]

In [ ]:
# Ask a question
query = "Tell me about the accursed tree, shajra e maloon as per hadees."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))      

In [ ]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

In [11]:
# Ask a question
query = "Tell me bout imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali also quote the entire incident."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that in the morning Amirul Mumineen (AS) came out and Abu Bakr and Umar were with him and a community of Muhajireen and Ansar were also with him until he reached Baqi and went and stood on a high place on the earth. When the sun arose, Alayhissalam said: “Salutation, O creature of Allah’s new creation who obeys Him.” Everyone heard a voice from the sky - an answer of someone who is answering: ‘And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O batin, O he who knows everything.’ When Abu Bakr, Umar, Muhajireen and Ansar heard what the sun said, they all fainted. Then they became conscious after a while, when Amirul Mumineen had left the place. They all went to the Holy Prophet (SAWA) with the community and said: “You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself.” [HADITH_NO:72]

The Holy Prophet (SAWA) asked: “What did you hear from it?” Everyone replied: “We heard it saying ‘Salutations be on you, O he who is Awwal.’” So he (SAWA) said: “It said truth. He is the first one who brought faith to me.” [HADITH_NO:72]

All said: “We heard it say: ‘O Aakheer’.” He replied: “It said truth, he is the last person of all who will separate from me. He will give me the ritual bath, shroud and will enter me in my grave.” [HADITH_NO:72]

All said: “We heard it say: ‘O Zahir’.” He replied: “It said truth. All my knowledge became Zahir (clear) for him.” [HADITH_NO:72]

All said: “We heard it say ‘O Batin’.” He replied: “It said truth. All my secrets are present with him.” [HADITH_NO:72]

All said: “We heard it say ‘O he, who knows everything’.” He replied: “It said truth. He knows what is lawful and unlawful, and compulsory, tradition and like it.” [HADITH_NO:72]

All stood up and said: “Muhammad put us in darkness.” and left from the [HADITH_NO:72]

References: [HADITH_NO:72]

In [12]:
# Ask a question
query = "Tell me about imam ali talking to the sun as per whatholy prophet toold him n how dud the sun reolied to imam ali also quote the entire incident."
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that in the morning Amirul Mumineen (AS) came out and Abu Bakr and Umar were with him and a community of Muhajireen and Ansar were also with him until he reached Baqi and went and stood on a high place on the earth. When the sun arose, Alayhissalam said: “Salutation, O creature of Allah’s new creation who obeys Him.” Everyone heard a voice from the sky - an answer of someone who is answering: ‘And salutation be on you as well, O that person, who is Awwal and O he, who is Aakheer, O Zahir, O batin, O he who knows everything.’ When Abu Bakr, Umar, Muhajireen and Ansar heard what the sun said, they all fainted. Then they became conscious after a while, when Amirul Mumineen had left the place. They all went to the Holy Prophet (SAWA) with the community and said: “You say that Ali is a man like us, when the sun addressed him in a manner that Allah has addressed Himself.”

The Holy Prophet (SAWA) asked: “What did you hear from it?” Everyone replied: “We heard it saying ‘Salutations be on you, O he who is Awwal.’ So he (SAWA) said: “It said truth. He is the first one who brought faith to me.”

All said: “We heard it say: ‘O Aakheer’.” He replied: “It said truth, he is the last person of all who will separate from me. He will give me the ritual bath, shroud and will enter me in my grave.”

All said: “We heard it say: ‘O Zahir’.” He replied: “It said truth. All my knowledge became Zahir (clear) for him.”

All said: “We heard it say ‘O Batin’.” He replied: “It said truth. All my secrets are present with him.”

All said: “We heard it say ‘O he, who knows everything’.” He replied: “It said truth. He knows what is lawful and unlawful, and compulsory, tradition and like it.”

All stood up and said: “Muhammad put us in darkness.” and left from there [HADITH_NO:72].

References: [HADITH_NO:72]

In [6]:
# Ask a question
query = "Tell me about What is the character of Zubair?"
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that Zubayr was outspoken and courageous. When Umar was on top of Zubayr, Zubayr said: "O son of Sahhak, By God, if my sword was in my hand, then you would not have got away from me." [HADITH_NO:4]

According to the traditions, Janabe Sulaym ibne Qays narrated that when Zubayr paid allegiance forcefully, he said to Umar: "O son of Sahhak, By God, if these evil people who supported you were not present then you would not able to come to me, and my sword would be with me, because I know your cowardice and disgrace, but you have got a few evil people from whom you gained strength and are attaching." [HADITH_NO:4]

According to the traditions, Janabe Sulaym ibne Qays narrated that Zubayr asked Umar: “Sahhak who? And can you stop me talking about Sahhak when Sahhak was a prostitute. Do you deny that? Was she no an Ethiopian slave of my grandfather Abdul Muttalib? Your granddfather Nufail committed adultery with her so your father Khattab was born. After adultery that slave girl was given to your grandfather by Abdul Muttalib, then your father was born, so he was my father’s slave who was born by adultery.” [HADITH_NO:4]

According to the traditions, Janabe Sulaym ibne Qays narrated that Imam Ali (AS) called Zubayr to him during the battle of Basra, saying: “O Abu Abdullah, come to me.” [HADITH_NO:29]

References: [HADITH_NO:4], [HADITH_NO:29]

In [8]:
# Ask a question
query = "Tell me about What was the character of Zubair in details"
res = qa_chain.invoke(query)
#print(res)
display(Markdown(res))

According to the traditions, Janabe Sulaym ibne Qays narrated that Zubayr was outspoken [HADITH_NO:4]. When Umar was on Zubayr's chest, Zubayr said: "O son of Sahhak, By God, if my sword was in my hand, then you would not have got away from me." He then paid allegiance [HADITH_NO:4].

According to the traditions, Janabe Sulaym ibne Qays narrated from Salman Farsi that after Zubayr paid allegiance, he said: “O son of Sahhak, By God, if these evil people who supported you were not present then you would not able to come to me, and my sword would be with me, because I know your cowardice and disgrace, but you have got a few evil people from whom you gained strength and are attaching.” [HADITH_NO:4].

Umar became very angry and said: “Are you talking about Sahhak?” Zubayr asked: “Sahhak who? And can you stop me talking about Sahhak when Sahhak was a prostitute. Do you deny that? Was she no an Ethiopian slave of my grandfather Abdul Muttalib? Your granddfather Nufail committed adultery with her so your father Khattab was born. After adultery that slave girl was given to your grandfather by Abdul Muttalib, then your father was born, so he was my father’s slave who was born by adultery.” Then Abu Bakr made peace between these two (Zubayr and Umar) and then both of them stopped quarrelling [HADITH_NO:4].

According to the traditions, Janabe Sulaym ibne Qays narrated from Abdullah ibn Abbas when Jabir ibn Abdullah Ansari was sitting next to him that Zubayr was let in to see the Holy Prophet (SAWA) because of Safiyya [HADITH_NO:61].

According to the traditions, Janabe Sulaym ibne Qays narrated that the Holy Prophet (SAWA) mentioned the incident of Jamal and the going of Ayeshah, Talhah and Zubayr [HADITH_NO:66].

References: [HADITH_NO:4], [HADITH_NO:61], [HADITH_NO:66]

## **LlamaIndex**

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader 
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
docs = SimpleDirectoryReader('pdf_docs').load_data()
docs


In [ ]:
# splitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter

In [ ]:
embeddings = HuggingFaceEmbedding(model_name="all-MiniLM-L12-v2")
embeddings

In [ ]:
storage_context = StorageContext.from_defaults(index_store=embeddings)
storage_context

In [ ]:
vector_store = VectorStoreIndex.from_documents(docs, storage_context=storage_context)
vector_store